# Acoustic Model Fine-Tuning

График картны хязгаарлагдмал байдлаас (гэрийн комьпютер) үүдэж whisper-tiny-г LoRA-гаар fine-tuning хийхээр шийдлээ.

Data-Preprocessing notebook дээрх train, test (80/20) датасетүүдийг ашиглан Whisper-г сургахдаа аль болох моделтойгоо tensor түвшинд харьцаж хийвэл дараа дараагийн алхамууд болох Speech LLM дээр хэрэгтэй гэж үзсний дор autotrainer зэрэг tool ашиглсангүй.

Acoustic model буюу Whisper-ийн (tiny) embedding хэмжээс 384 юм байна. Whisper-ээс гарах гаралтыг LLM-н хүлээж авах (shape) хэмжээс рүү хөрвүүлэхдээ би зөвхөн шугаман (linear) layer-ууд ашиглсан. Энийг чухам яагаад гээд судлаад үзэхэд, 2021-онд Transformer архтектурыг авианы датад хэрэглсэн [Hubert-ын](https://arxiv.org/abs/2106.07447) өгүүллэгт Acoustic model-ын embedding-гээс натурал хэлний текст мэдээллийг гаргахдаа, зөвхөн шугаман функц ашиглаж харуулсан байж.

In [1]:
import torch
import torchaudio
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from transformers import WhisperForConditionalGeneration, WhisperProcessor
from peft import LoraConfig, get_peft_model

In [2]:
class CVMNDataset(Dataset):
    def __init__(self, tsv_path, wav_dir, encoder):
        self.df = pd.read_csv(tsv_path, delimiter='\t')
        self.wav_dir = wav_dir
        self.encoder = encoder
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        wav, sr = torchaudio.load(f"{self.wav_dir}/{row['path']}")
        
        feats = self.encoder(wav.squeeze().numpy(), sampling_rate=16000, return_tensors="pt").input_features[0]
        tokenized_text = self.encoder.tokenizer(row["sentence"], truncation=True, max_length=448).input_ids
        
        return {
            "input_features": feats,
            "labels": torch.tensor(tokenized_text)
        }
    
    def __len__(self):
        return len(self.df)

model_name = "openai/whisper-tiny"
device = "cuda"

# Load processor
processor = WhisperProcessor.from_pretrained(model_name)

# Load model in FP16
model = WhisperForConditionalGeneration.from_pretrained(
    model_name,
    torch_dtype=torch.float16
).to(device)

# Configure LoRA
lora_config = LoraConfig(
    r=16, 
    lora_alpha=32, 
    target_modules=["q_proj", "v_proj", "k_proj", "out_proj", "fc1", "fc2"],  # Which layers to adapt
    lora_dropout=0.05,
    bias="none",
)

model = get_peft_model(model, lora_config)

model.print_trainable_parameters()

model.train()

Loading weights:   0%|          | 0/167 [00:00<?, ?it/s]

trainable params: 1,081,344 || all params: 38,841,984 || trainable%: 2.7840


PeftModel(
  (base_model): LoraModel(
    (model): WhisperForConditionalGeneration(
      (model): WhisperModel(
        (encoder): WhisperEncoder(
          (conv1): Conv1d(80, 384, kernel_size=(3,), stride=(1,), padding=(1,))
          (conv2): Conv1d(384, 384, kernel_size=(3,), stride=(2,), padding=(1,))
          (embed_positions): Embedding(1500, 384)
          (layers): ModuleList(
            (0-3): 4 x WhisperEncoderLayer(
              (self_attn): WhisperAttention(
                (k_proj): lora.Linear(
                  (base_layer): Linear(in_features=384, out_features=384, bias=False)
                  (lora_dropout): ModuleDict(
                    (default): Dropout(p=0.05, inplace=False)
                  )
                  (lora_A): ModuleDict(
                    (default): Linear(in_features=384, out_features=16, bias=False)
                  )
                  (lora_B): ModuleDict(
                    (default): Linear(in_features=16, out_features=384, bias=False)

In [3]:
dataset = CVMNDataset(
    "train.tsv",
    "./wav_16k",
    processor
)


def collate_whisper(batch):
    feats = torch.stack([b["input_features"] for b in batch])
    labels = pad_sequence(
        [b["labels"] for b in batch], 
        batch_first=True, 
        padding_value=-100
    )
    return {"input_features": feats, "labels": labels}
    
loader = DataLoader(
    dataset, 
    batch_size=4,
    shuffle=True,
    collate_fn=collate_whisper,
)

import gc
gc.collect()

40

In [14]:
for epoch in range(0,10):
    total_loss = 0
    optimizer.zero_grad()
    
    for i, batch in enumerate(loader):
        input_features = batch["input_features"].to(device, dtype=torch.float16)
        labels = batch["labels"].to(device)
        
        # Forward pass
        outputs = model(
            input_features=input_features,
            labels=labels
        )
        
        loss = outputs.loss / accumulation_steps
        loss.backward()
        
        if (i + 1) % accumulation_steps == 0:
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            optimizer.zero_grad()
            torch.cuda.empty_cache()
        
        total_loss += loss.item()
        del input_features, labels, outputs
    
    if len(loader) % accumulation_steps != 0:
        optimizer.step()
        optimizer.zero_grad()
    
    avg_epoch_loss = (total_loss / len(loader)) * accumulation_steps
    print(f"Epoch {epoch} completed | avg_loss = {avg_epoch_loss:.4f}")


    model.eval()
    total_test_loss = 0.0
    num_batches = 0
    with torch.no_grad():
        for batch in test_loader:
            input_features = batch["input_features"].to(device, dtype=torch.float16)
            labels = batch["labels"].to(device)

            outputs = model(
                input_features=input_features,
                labels=labels
            )

            total_test_loss += outputs.loss.item()
            num_batches += 1
    
            del input_features, labels

    avg_test_loss = total_test_loss / num_batches
    print( f"Epoch {epoch} test_loss = {avg_test_loss:.4f}")
        
    model.train()
    torch.cuda.empty_cache()

Epoch 6 completed | avg_loss = 1.2743
Epoch 6 test_loss = 1.4798
Epoch 7 completed | avg_loss = 1.2000
Epoch 7 test_loss = 1.4524
Epoch 8 completed | avg_loss = 1.1192
Epoch 8 test_loss = 1.4963
Epoch 9 completed | avg_loss = 1.0557
Epoch 9 test_loss = 1.4198


In [16]:
model.save_pretrained("whisper_mn_epoch_10")
processor.save_pretrained("whisper_mn_epoch_10")

['whisper_mn_epoch_10/processor_config.json']

In [27]:
import torch
import torchaudio
from transformers import WhisperForConditionalGeneration, WhisperProcessor

# 1. Load Model correctly (use torch_dtype)
model = WhisperForConditionalGeneration.from_pretrained(
    model_path, 
    torch_dtype=torch.float16
).to("cuda")


processor = WhisperProcessor.from_pretrained(model_path)
test_df = pd.read_csv("test.tsv", sep="\t")
samples = test_df.sample(n=5)
model.eval()

for i, row in samples.iterrows():

    audio_path = f'./wav_16k/{row["path"]}'

    speech, sr = torchaudio.load(audio_path)

    assert(sr) == 16000
    
    display(Audio(speech.squeeze().numpy(), rate=16000))
    inputs = processor(
    speech.squeeze().numpy(),
    sampling_rate=16000,
    return_tensors="pt"
)

    input_features = inputs.input_features.to("cuda", dtype=torch.float16)
    with torch.no_grad():
        generated_ids = model.generate(
            input_features, 
            language="mongolian", 
            task="transcribe"
        )
        
    prediction = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
    print(f"Ground Truth: {row['sentence']}")
    print(f"Prediction: {prediction}\n")

Loading weights:   0%|          | 0/167 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/128 [00:00<?, ?it/s]

Ground Truth: Илүү ч учраас тэд агнасан анг нь өдөр дараалан булааж авч чадсан хэрэг.
Prediction: Элүүч Өчрөөс хатийн даггансан хангүйн өдөр дараа халан боллааж авч авчаахын хэрэг.



Ground Truth: Аржилстритийн эргэн тойронд нь цагдаагийн хамгаалалт гарч хатан хааны гэр бүлийнхэн морилон ирэв.
Prediction: Алжисрээтийн нь эргэн торин тогдаагийн хамгаалдгаарч, атан хааныг гэр бүлийн хэн марилгэн дэрэв.



Ground Truth: Намайг орлогчийн хугацаанд зөрж чадахгүй гээд зориуд томилж байгаа юм.
Prediction: Намайг орложчин ухцаан зөр чадхүүхүй гээд зорьвд өмрөж байгаа юм.



Ground Truth: Улсад нэг хуанху байх бөгөөд Улсын эх гэсэн хүндэтгэлийн цолтойгоор гол ордонд заларч асан.
Prediction: Усанх нийг хонхоо байх бүгүйл үлсийн ир гэсэн хүнд түлгийн солтойгаар голоордонざалаар часан.



Ground Truth: Орчин үеийн мэдлэг боловсрол бүхий боловсон хүчинтэй, зохион байгуулалтын хувьд бусдыгаа төлөөлж чадахуйц юм.
Prediction: Орчүүрийн мэдлэг болоо сурлахүй болоосон хичийнтэй, цухан байгаалтой хөд бустайгаа төлөөл ч чадаггойц юм.

